# Urban Heat Index

By Cascade Tuholske 2020.01.21

Notebook is designed to take areal-averaged CHIRTS Tmax for each GHS-UCDB and down-scaled MERRA-2 humidity data and calculate the heat index for each city with a Tmax >80F.

[NOAA Heat Index Equation](https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml)

**THINGS I HAVE Coded**
- C to F function
- Rothfusz regression and adjustments
- Steadman's

**NOTES**
At first I thought the heat index values for the hottest areas were insane (> 140F), but I spot checked the results and the [NOAA Heat Index Table](https://www.kjrh.com/weather/weather-blog-what-exaclty-is-the-heat-index) simply reds out values where Tmax >40C and RH > 50%. So I guess were are on track ...

#### Dependencies

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian
import math

## Functions

In [2]:
def C_to_F(Tmax_C):
    "Function converts temp in C to F"
    Tmax_F = (Tmax_C * (9/5)) + 32
    
    return Tmax_F

In [3]:
def F_to_C(Tmax_F):
    "Function converts temp in F to C"
    Tmax_C = (Tmax_F - 32) * (5/9)
    
    return Tmax_C

In [4]:
def csv_to_xr(file_in, time_dim, space_dim):
    
    """ Function reads in a csv w/ GHS-UCDB IDs and temp, isolates the temp
    and returns a xarray data array with dims set to city ids and dates
    
    Args:
        file_in = file name and path
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    
    df = pd.read_csv(file_in) # read the file in as a df
    print(df.shape)
    
    df_id = df[space_dim] # get IDs
    df = df.iloc[:,3:] # get only temp columns
    df.index = df_id # set index values
    df_drop = df.dropna() # Drop cities w/ no temp record 
    print(len(df_drop))
    
    arr = df_drop.to_numpy() # turn temp cols into an np array
    
    # make xr Data Array w/ data as temp and dims as spece (e.g. id)
    
    # Note 2019 09 17 changed to xr.Dataset from xr.Dataarray
    xr_da = xr.DataArray(arr, coords=[df_drop.index, df_drop.columns], 
                            dims=[space_dim, time_dim])
    return xr_da

In [5]:
def heatindex(Tmax, RH, unit_in, unit_out):
    
    """Make Heat Index from 2m air and relative humidity following NOAA's guidelines: 
    https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml. It is assumed that the
    tempatures and RH are geographically and temporally aligned in the x-arrays and can be stacked
    to the funciton.
    
    --- update as needed cpt 2020.02.17
    
    Args:
        Tmax = x-array of tempatures
        RH = x-array of realtive humitity
        unit_in = F or C, will convert C to F to apply heat index
        unit_out = If C is desired, will convert data to C
        
    Returns HI
    """
    
    # Make all data as float
    Tmax = Tmax.astype('float')
    RH = RH.astype('float')
    
    # 1 convert C to F if needed
    if unit_in == 'C':
        Tmax = C_to_F(Tmax)
        
    # 2 Apply Steadman's and average with Tmax
    USE_STEADMAN = (0.5 * (Tmax + 61.0 + ((Tmax-68.0)*1.2) + (RH*0.094)) + Tmax) / 2 < 80
    STEADMAN = USE_STEADMAN * (0.5 * (Tmax + 61.0 + ((Tmax-68.0)*1.2) + (RH*0.094))) #.astype(int)
    
    # 3 Use Rothfusz if (STEADMAN + Tmax) / 2 > 80
    USE_ROTH = (0.5 * (Tmax + 61.0 + ((Tmax-68.0)*1.2) + (RH*0.094)) + Tmax) / 2 > 80
    ROTH = USE_ROTH * (-42.379 + 2.04901523*Tmax + 10.14333127*RH - .22475541*Tmax *RH - .00683783*Tmax*Tmax - .05481717*RH*RH + .00122874*Tmax*Tmax*RH + .00085282*Tmax*RH*RH - .00000199*Tmax*Tmax*RH*RH)

    # 3 Adjust Roth 1
    USE_ADJ1 = (RH < 13) & (Tmax > 80) & (Tmax < 112)
    ADJ1_RH = USE_ADJ1 * RH #.astype(int)
    ADJ1_RH = ADJ1_RH.where(ADJ1_RH != 0) #ADJ1_RH[ADJ1_RH == 0] = np.nan
    ADJ1_Tmax = USE_ADJ1 * Tmax # .astype(int)
    ADJ1_Tmax = ADJ1_Tmax.where(ADJ1_Tmax != 0) #ADJ1_Tmax[ADJ1_Tmax == 0] = np.nan
    ADJ1 = ((13-ADJ1_RH)/4)*np.sqrt((17-abs(ADJ1_Tmax-95.))/17)
    ADJ1 = np.nan_to_num(ADJ1, 0)
    
    ADJ1_ROTH = ROTH * USE_ADJ1
    ADJ1_ROTH = ADJ1_ROTH - ADJ1
    
    # 4 Adjust Roth 2
    USE_ADJ2 = (RH > 85) & (Tmax > 80) & (Tmax < 87)
    ADJ2_RH = USE_ADJ2 * RH #.astype(int)
    ADJ2_RH = ADJ2_RH.where(ADJ2_RH != 0) #ADJ2_RH[ADJ2_RH == 0] = np.nan
    ADJ2_Tmax = USE_ADJ2.astype(int) * Tmax
    ADJ2_Tmax = ADJ2_Tmax.where(ADJ2_Tmax != 0) #ADJ2_Tmax[ADJ2_Tmax == 0] = np.nan
    ADJ2 = ((ADJ2_RH-85)/10) * ((87-ADJ2_Tmax)/5)
    ADJ2 = np.nan_to_num(ADJ2, 0)
    
    ADJ2_ROTH = ROTH * USE_ADJ2
    ADJ2_ROTH = ADJ2_ROTH + ADJ2
    
    # Roth w/o adjustments
    ROTH = ROTH * ~USE_ADJ1 * ~USE_ADJ2
    
    # sum the stacked arrays
    HI = ROTH + STEADMAN + ADJ1_ROTH +  ADJ2_ROTH 
    
    # Convert HI to C if desired
    if unit_out == 'C':
        HI = F_to_C(HI)
    
    # return for test
    # return STEADMAN, ADJ1_ROTH, ADJ2_ROTH, ROTH, HI
    
    return HI

In [28]:
def apply_heatindex(DIR_Tmax, DIR_RH, DIR_HI, unit_in, unit_out):
    """Function applies NOAA's heatindex to two pair directories w/ CSVs of realitive humidity
    and tempatures, respective, in a pairwise fashion
    
    Args:
        DIR_Tmax = the directory where Tmax .csv files are stored
        DIR_RH = the directory where RH .csv files are stored
        DIR_HI = the directory where HI files will be written
        unit_in = temp unit for Tmax (C or F)
        unit_out = desired temp unit for HI (C or F) for the output
    """
    Tmax_fn_list = glob.glob(DIR_Tmax+'*.csv')
    RH_fn_list = glob.glob(DIR_RH+'*.csv')

    for Tmax_fn, RH_fn in zip(sorted(Tmax_fn_list),sorted(RH_fn_list)):
    
        # Check the years RH and Tmax 
        Tmax_year = Tmax_fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        print('Tmax year is ',Tmax_year)
        RH_year = RH_fn.split('GHS-Tmax-RH_')[1].split('.csv')[0]
        print('RH year is ', RH_year)

        # Read csv as x-array
        Tmax_xr = csv_to_xr(Tmax_fn, time_dim = 'date', space_dim = 'ID_HDC_G0')
        RH_xr = csv_to_xr(RH_fn, time_dim = 'date', space_dim = 'ID_HDC_G0')

        # Make heat index
        hi = heatindex(Tmax_xr, RH_xr, unit_in = unit_in, unit_out = unit_out)

        # Get countries and city ids
        df_out = pd.read_csv(Tmax_fn)
        df_out = df_out[['ID_HDC_G0', 'CTR_MN_NM']]

        # write to csv
        hi_df = hi.to_pandas()
        #hi_df['ID_HDC_G0'] = hi_df.index
        df_out = df_out.merge(hi_df, on = 'ID_HDC_G0', how = 'inner')
        df_out_nm = 'GHS-HI-DAILY_'+Tmax_year+'.csv'
        df_out.to_csv(DIR_HI+df_out_nm)
        print(RH_year, ' done \n')
    
    print('ALL DONE!')

# Run it

In [29]:
DIR_Tmax = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-Tmax/'
DIR_RH = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-RH/'
DIR_HI = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-HI/'

# FIRE HI
apply_heatindex(DIR_Tmax, DIR_RH, DIR_HI, unit_in = 'C', unit_out = 'C')

Tmax year is  1983
RH year is  1983
(13135, 368)
13067
(13135, 368)
13067
1983  done 

Tmax year is  1984
RH year is  1984
(13135, 369)
13067
(13135, 369)
13067
1984  done 

Tmax year is  1985
RH year is  1985
(13135, 368)
13067
(13135, 368)
13067
1985  done 

Tmax year is  1986
RH year is  1986
(13135, 368)
13067
(13135, 368)
13067
1986  done 

Tmax year is  1987
RH year is  1987
(13135, 368)
13067
(13135, 368)
13067
1987  done 

Tmax year is  1988
RH year is  1988
(13135, 369)
13067
(13135, 369)
13067
1988  done 

Tmax year is  1989
RH year is  1989
(13135, 368)
13067
(13135, 368)
13067
1989  done 

Tmax year is  1990
RH year is  1990
(13135, 368)
13067
(13135, 368)
13067
1990  done 

Tmax year is  1991
RH year is  1991
(13135, 368)
13067
(13135, 368)
13067
1991  done 

Tmax year is  1992
RH year is  1992
(13135, 369)
13067
(13135, 369)
13067
1992  done 

Tmax year is  1993
RH year is  1993
(13135, 368)
13067
(13135, 368)
13067
1993  done 

Tmax year is  1994
RH year is  1994
(13135,

# Test

In [ ]:
# Test with the conditions above test
temp = xr.DataArray([70, 100, 85, 100])
rh = xr.DataArray([5, 10, 90, 20])
a,b,c,d, e = heatindex(temp, rh, unit_in = 'F', unit_out = 'F')
print('Stead ', a)
print('Adj1', b)
print('Adj2', c)
print('Roth', d)
print('All HI values', e)

In [ ]:
# Test with actual data from HI chart : https://www.weather.gov/safety/heat-index
# Results fit the table

temp = xr.DataArray([80, 82, 94, 104])
rh = xr.DataArray([40, 40, 40, 40])
a,b,c,d, e = heatindex(temp, rh, unit_in = 'F', unit_out = 'F')
print(rh[0], temp[0], a)
print(rh[0], temp[1], b)
print(rh[0], temp[2], c)
print(rh[0], temp[3], d)
print('All HI values', e)

# Test With CSVs

In [7]:
DIR_RH = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-RH/'
DIR_Tmax = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-Tmax/'
DIR_HI = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-HI/'

In [8]:
FN_RH = 'GHS-Tmax-RH_1984.csv'
FN_Tmax = 'GHS-Tmax-DAILY_1984.csv'

In [9]:
RH = csv_to_xr(DIR_RH+FN_RH, space_dim = 'ID_HDC_G0', time_dim = 'date')
Tmax = csv_to_xr(DIR_Tmax+FN_Tmax, space_dim = 'ID_HDC_G0', time_dim = 'date')

(13135, 369)
13067
(13135, 369)
13067


In [14]:
ugh = pd.read_csv(DIR_RH+FN_RH)

In [16]:
ugh.head()

,Unnamed: 0,ID_HDC_G0,CTR_MN_NM,1984.01.01,1984.01.02,1984.01.03,1984.01.04,1984.01.05,1984.01.06,1984.01.07,...,1984.12.22,1984.12.23,1984.12.24,1984.12.25,1984.12.26,1984.12.27,1984.12.28,1984.12.29,1984.12.30,1984.12.31
0,0,5782,Russia,73.26125,95.588640,82.617790,98.068370,85.623245,85.803970,93.096270,...,89.103000,91.507324,83.534970,83.701180,86.040480,100.000000,100.000000,86.380646,84.141860,84.683655
1,1,3316,Russia,78.29920,77.267720,75.377580,71.599396,72.656975,83.402626,87.227480,...,87.097860,83.475450,76.018845,90.875305,88.562870,86.980260,80.649250,71.785830,86.918040,92.814255
2,2,5645,Russia,75.45298,80.180900,71.191290,76.220985,74.785530,75.802470,85.675850,...,83.241470,99.460300,86.276450,80.986660,86.651370,87.038666,100.000000,58.798153,74.991400,75.359690
3,3,3185,Finland,86.57464,100.000000,96.645905,88.796890,93.723660,92.737060,88.126305,...,92.221825,94.210785,97.585330,98.801636,96.193380,92.674510,93.881500,90.576410,88.716830,94.450800
4,4,3539,Russia,82.58847,83.099075,84.778280,85.860880,84.793810,91.218970,88.119606,...,78.398300,84.420200,82.888150,87.185104,78.806526,84.010660,87.359116,86.407010,83.318504,93.662796


In [10]:
test = heatindex(Tmax, RH, unit_in = 'C', unit_out = 'C')

In [11]:
test_pd = test.to_pandas()

In [12]:
test_pd

date,1984.01.01,1984.01.02,1984.01.03,1984.01.04,1984.01.05,1984.01.06,1984.01.07,1984.01.08,1984.01.09,1984.01.10,...,1984.12.22,1984.12.23,1984.12.24,1984.12.25,1984.12.26,1984.12.27,1984.12.28,1984.12.29,1984.12.30,1984.12.31
ID_HDC_G0,,,,,,,,,,,,,,,,,,,,,
5782,-19.603278,-16.509366,-17.688222,-15.782093,-20.513494,-20.024169,-19.646038,-20.865014,-23.657549,-23.093398,...,-23.028032,-23.353347,-29.363638,-30.545672,-25.763434,-12.875356,-19.427420,-38.983521,-37.406984,-38.011216
3316,-17.271016,-15.895727,-15.972710,-19.058956,-8.680372,-7.762624,-12.484643,-11.473094,-14.277118,-16.592355,...,-1.660659,-6.635546,-1.218867,0.262298,-0.362172,-4.487592,-7.101775,-11.484543,-3.092274,-1.625348
5645,-13.006575,-13.387874,-16.111549,-16.823017,-16.055264,-10.907404,-11.638051,-20.410790,-18.092921,-18.279321,...,-14.435531,-12.028642,-17.997921,-18.128609,-7.626927,-11.732642,-11.622293,-28.838632,-38.556875,-38.875387
3185,-12.230020,-6.028401,-5.054383,-11.613501,-9.214698,-12.086568,-17.891322,-19.904414,-20.389427,-17.960100,...,-1.138911,-2.682360,0.229404,0.537775,-3.693834,-8.996114,-8.323893,-8.845587,-5.864510,-5.683066
3539,-14.200230,-11.768106,-6.437049,-5.531272,-6.381153,-9.047180,-4.707061,-11.671482,-12.853189,-14.914830,...,-9.467633,-10.850993,-11.458981,-7.923813,-13.172307,-10.326792,-6.703562,-7.223617,-8.078176,-6.381526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13046,17.613376,22.058742,24.395478,25.647603,13.644481,14.314127,18.348333,23.937890,24.927829,17.254387,...,15.221688,18.797521,20.291237,17.869695,17.951916,18.895673,16.726665,18.384778,23.172863,23.966231
1116,22.075079,16.267125,12.688323,19.627985,17.939651,17.380007,17.910172,12.984902,16.819428,16.627256,...,23.879452,24.191684,24.259476,26.468886,27.553832,21.417997,25.000973,27.329142,17.769248,18.438768
1114,14.621860,11.978608,10.111684,11.588975,13.347414,13.904955,12.043949,9.579040,12.948700,11.529167,...,19.486219,20.325655,20.207977,22.711821,20.904695,15.772708,18.978708,17.572250,13.807125,14.189227


In [ ]:
test_pd['1984.07.01'].sort_values(ascending = False).head(50)

# Build Dir Walk

In [ ]:
# Get files from both RH and temp

DIR_Tmax = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-Tmax/'
DIR_RH = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-RH/'
DIR_HI = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-HI/'

print(DIR_Tmax)
print(DIR_RH) 

Tmax_fn_list = glob.glob(DIR_Tmax+'*.csv')
RH_fn_list = glob.glob(DIR_RH+'*.csv')

for Tmax_fn, RH_fn in zip(sorted(Tmax_fn_list),sorted(RH_fn_list)):
    
    # Check the years RH and Tmax 
    Tmax_year = Tmax_fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
    print('Tmax year is ',Tmax_year)
    RH_year = RH_fn.split('GHS-Tmax-RH_')[1].split('.csv')[0]
    print('RH year is ', RH_year)
    
    # Read csv as x-array
    Tmax_xr = csv_to_xr(Tmax_fn, time_dim = 'date', space_dim = 'ID_HDC_G0')
    RH_xr = csv_to_xr(RH_fn, time_dim = 'date', space_dim = 'ID_HDC_G0')
    
    # Make heat index
    hi = heatindex(Tmax_xr, RH_xr, unit_in = 'C', unit_out = 'F')
    

    # CASCADE GO LOOK AT HOW X-ARRAYS ARE WRITTEN TO CSVS IN EARLIER CODE <<<<---- 
    
    # write to csv
    df = hi.to_pandas()
    df_out_nm = 'GHS-HI-DAILY_'+Tmax_year+'.csv'
    df.to_csv(DIR_HI+df_out_nm)
    print(RH_year, ' done \n')

In [ ]:
df.head()

# Check HI

In [ ]:
df.head()

In [ ]:
df.index

In [ ]:
# get GHS-UCDB
ghs = gpd.read_file('/home/cascade/projects/UrbanHeat/data/raw/GHS_UCDB/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')

In [ ]:
out = gpd.GeoDataFrame()
out['geometry'] = ghs['geometry']
out['ID_HDC_G0'] = ghs['ID_HDC_G0']

HI_out = gpd.GeoDataFrame()
HI_out['ID_HDC_G0'] = df.index
HI_out['2016.07.01'] = df['2016.07.01']

out = pd.merge(out, HI_out, on = 'ID_HDC_G0', how = 'inner')

In [ ]:
#out.to_file('/home/cascade/projects/UrbanHeat/HI_20160701_test.shp')

In [ ]:
# Check hotcheck values 

Tmax =  pd.read_csv(Tmax_fn)
RH = pd.read_csv(RH_fn)

check = pd.DataFrame()
check['2016.07.01.HI'] = df['2016.07.01'].sort_values(ascending = False).head(50)
check = check.merge(Tmax[['ID_HDC_G0', '2016.07.01']], on = 'ID_HDC_G0', how = 'inner')
check.rename(columns = {'2016.07.01':'2016.07.01.Tmax'}, inplace = True)
check = check.merge(RH[['ID_HDC_G0', '2016.07.01']], on = 'ID_HDC_G0', how = 'inner')
check.rename(columns = {'2016.07.01':'2016.07.01.RH'}, inplace = True)
# check['2016.07.01.RH'] = RH['2016.07.01'].sort_values(ascending = False).head(50)


# df['2016.07.01'].sort_values(ascending = False).head(50)

In [ ]:
check.head()

In [ ]:
checkTmax = xr.DataArray(check['2016.07.01.Tmax'].to_numpy())
checkRH = xr.DataArray(check['2016.07.01.RH'].to_numpy())


In [ ]:
# Now check the heat index
checkHI = heatindex(checkTmax, checkRH, unit_in = 'C', unit_out = 'F')

In [ ]:
checkHI

In [ ]:
# Test with the conditions above test
temp = xr.DataArray([42, 41, 40, 39])
rh = xr.DataArray([52, 52, 52, 52])
heatindex(temp, rh, unit_in = 'C', unit_out = 'F')


# Well it makes sense

These crazy high values make sense when checked with NOAA's table. But once you have a Tmax >40C and RH > 40%, the HI values is crazy high ...

# OLD CODE

In [ ]:
def csv_to_xr(file_in, time_dim, space_dim):
    
    """ Function reads in a csv w/ GHS-UCDB IDs and temp, isolates the temp
    and returns a xarray data array with dims set to city ids and dates
    
    Args:
        file_in = file name and path
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    
    df = pd.read_csv(file_in) # read the file in as a df
    print(df.shape)
    
    df_id = df[space_dim] # get IDs
    df = df.iloc[:,3:] # get only temp columns
    df.index = df_id # set index values
    df_drop = df.dropna() # Drop cities w/ no temp record 
    print(len(df_drop))
    
    arr = df_drop.to_numpy() # turn temp cols into an np array
    
    # make xr Data Array w/ data as temp and dims as spece (e.g. id)
    
    # Note 2019 09 17 changed to xr.Dataset from xr.Dataarray
    temp_xr_da = xr.DataArray(arr, coords=[df_temp_drop.index, df_temp_drop.columns], 
                            dims=[space_dim, time_dim])
    
    return xr_da

#### #2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city

In [ ]:
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    dayTot_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # this is actually getting the total events of all, 2019-09-22
        dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    df_out['total_days'] = dayTot_list
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

#### #3 Function splits the dataset into Tmax events (continuous days >Tmax) for each city

In [ ]:
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax, total_days):
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        country: country for each city as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    tot_days = total_days
    
    # lists to fill
    city_id_list = []
    tot_days_list = []
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        
        # fill lists
        city_id_list.append(city_id)
        tot_days_list.append(tot_days)
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['total_days'] = tot_days_list
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### #4 Function feeds output from function 2 into function 3

In [ ]:
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats
    
        NOTE - If you add arguments to event_split to make more states, 
        be sure to update this function
    
        args:
            df: input dataframe
        
    """
    df_out = pd.DataFrame()
    
    # NOTE - If you add arguments to event_split to make more stats, 
    # be sure to update this function
    
    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        total_days = row['total_days'] # get total number of tmax days

        df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)

        df_out = df_out.append(df)
    
    return df_out

#### #5 Loops through a file list and applies functions 1 - 4 to the data to produce Tmax stats for all tmax events in a given year across all cities in the dataset

In [ ]:
def stats_loop(dir_in, dir_out, fn_out, time_dim, space_dim, Tthresh):
    
    """ Loop through a dir with csvs to apply csv_to_xr and
    tmax_stats function and save out a .csv for each year
    
    Args:
        dir_in = dir path to loop through
        dir_out = dir path to save files out
        fn_out = string to label out files
        time_dim = name for time dim as a str ... use date :-) for csv_to_xr function
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0) for csv_to_xr function
        Tthresh = int of temp threshold for temp_event function -- 40.6 is used

    """
    
    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
        
    # Git File list
    fn_list = glob.glob(dir_in+'*.csv')
    
    for fn in sorted(fn_list):
        
        # Get year for arg for temp_event function
        year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        print(year)
        
        # read csv as a data array
        temp_xr_da = csv_to_xr(fn, time_dim, space_dim)
        
        # data array to tmax events, out as df
        df_days = tmax_days(temp_xr_da, Tthresh)
        
        # tmax events stats, out as df
        df_out = tmax_stats(df_days)
        
        # merge to get countries
        ghs_ids_df_out = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'inner') 
        
        # write it all out
        ghs_ids_df_out.to_csv(dir_out+fn_out+year+'.csv')

        print(year, 'SAVED!')

#### Run Code

In [ ]:
dir_in = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/' # ghs ID list
dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
fn_out = 'CHIRTS-GHS-Events-Stats'
time_dim = 'date'
space_dim = 'ID_HDC_G0'
Tthresh = 40.6


In [ ]:
#stats_loop(dir_in, dir_out, fn_out, time_dim, space_dim, Tthresh)

## QA/QC

In [ ]:
# Load files

dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
fn_out = 'CHIRTS-GHS-Events-Stats'

In [ ]:
qc_df = pd.read_csv(dir_out+fn_out+'1983.csv')

In [ ]:
qc_df.head()

In [ ]:
dc_city = qc_df[qc_df['ID_HDC_G0'] == 5534]

In [ ]:
dc_city

## Testing

In [ ]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/data_out_urbanheat/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out/'

In [ ]:
# File name to test
fn_in = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
# Open a raw file
xr1983 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

In [ ]:
days1983 = tmax_days(xr1983, 40.6)

In [ ]:
test = days1983[0:30]
test

# Maybe add in days_total

In [ ]:
# Build routine for loop through a csv

df_out = pd.DataFrame()

for index, row in test.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days
    
    df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)
    
    df_out = df_out.append(df)

In [ ]:
df_out

# Fixing code
2019.10.19 Cascade Tuholkse

Need to fix ```event split``` function

Somewhere in 1984 is this event sequence: ['1984.01.01' '1984.01.02' '1984.01.07']


**FOUND PROBLEM AND IT IS FIXED**

In [ ]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/data_out_urbanheatv/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out_urbanheat/testout/'

In [ ]:
# File name to test
fn_in = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
# Open a raw file
xr1983 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

In [ ]:
# Find all the tmax days
tmax1983 = tmax_days(xr1983, 40.6)

In [ ]:
# Make a subset

test = tmax1983[tmax1983['ID_HDC_G0'] == 6279]
test

In [ ]:
for index, row in test.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days



In [ ]:
# Convert to jul days
jul_days = pd.to_datetime(dates).to_julian_date()
jul_days


In [ ]:
mjd = 2445507.5 - 43200
dt = julian.from_jd(mjd, fmt='mjd')
print(dt)


In [ ]:
dates

In [ ]:
dates = ['1983.06.20', '1983.06.23', '1983.06.24', '1983.06.25',
        '1983.06.26', '1983.06.27', '1983.06.28', '1983.06.29',
        '1983.06.30', '1983.07.01', '1983.07.21', '1983.07.22',
        '1983.07.23', '1983.08.01']

pd_dates = pd.to_datetime(dates)
df_dates = pd.DataFrame()
df_dates['dates'] = pd_dates



test = df_dates['dates'].apply(lambda x: x.toordinal())

In [ ]:
from datetime import date
date.fromordinal(test[1])

In [ ]:
counter = 0

In [ ]:
event_dates = dates[0:10] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

event_dates = dates[10:13] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

event_dates = dates[13:14] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

In [ ]:
event_dates_list = []
counter = 0
start = 0
end = 0

for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
    
    len_dates = len(dates) # len of all Tmax dates for a given city
#   print(len(dates))
    
    seq = list(map(itemgetter(1), g)) # isolate seq. days
    dur = len(seq) # duration of each event 
    
    counter = counter + dur
    end = counter
    
    print(end)
    
    event_dates = dates[start:end] # dates of tmax days during each event
    print(event_dates)
    
    start = counter
    
# #     print('dur= ', dur)
    
#     event_dates = dates[0:10] # dates of tmax days during each event
#     print(event_dates)
    
#     event_dates = dates[11:13] # dates of tmax days during each event
#     print(event_dates)
    
#     event_dates = dates[:len_dates] # dates of tmax days during each event
#     print(event_dates)
    
    
    
    
#     counter = counter + dur

#     print('counter = ', counter)
    
#     dif = dur - counter
#     print('dif = ', dif)
    
#     start = start 
#     print('start = ',start)
    
# #     end = counter + dur
# #     print("start = ",end)
    
    
    
#     counter = counter + dur
#     print(counter)
#     end = counter + dur
#     start
#     event_dates = dates[start:end] # dates of tmax days during each event
#     print(event_dates)
# #     intense = intensity[0:dur] # intensity of each day during event
#     temp = tmax[0:dur] # temp of each day during event
#     avg_temp = mean(temp) # avg. temp during event
#     avg_int = mean(intense) # avg. intensity during event
#     tot_int = intense.sum() # total intensity during event
    
#     event_dates_list.append(event_dates)
    


In [ ]:
event_dates_list

In [ ]:
df = pd.DataFrame()
df['event_dates'] = event_dates_list
df

In [ ]:
import more_itertools as mit

In [ ]:
# run tmax stats ------> CLEARLY NOTE WORKING

# tmax1983_sub_stats = tmax_stats(tmax1983_sub)

In [ ]:
type(tmax1983['dates'][1])

In [ ]:
dates = tmax1983_sub['dates']
#jul_days = pd.to_datetime(tmax1983['dates']).to_julian_date()

In [ ]:
# Try to break up events

for index, row in tmax1983_sub.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days

#     df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)

#     df_out = df_out.append(df)

dates

In [ ]:
type(df_out['events'][0])

#### Test Heat Index

Get some paired values from the [NOAA CHART](https://www.weather.gov/safety/heat-index) and test the output. 

These results seem to be working well, though 104 F and 60 RH produced a higher number than the chart ...

In [ ]:
temp = [25, 27, 35, 40, 41] # temp C
RH = 60
test = pd.DataFrame()
test['temp'] = temp
test['RH'] = RH

In [ ]:
for i, row in test.iterrows():
    out = make_hi(row['temp'], row['RH'])
    print('\n')

In [ ]:
# turn the temp x array from C to F

temp_xr_da = C_to_F(temp_xr_da)

In [ ]:
# apply steadman's

HI_steadman = steadman_hi(temp_xr_da, RH_xr_da)

In [ ]:

## this doesn't work 
#HI_rothfusz = rothfusz_hi(HI_steadman, temp_xr_da, RH_xr_da)

In [ ]:
def C_to_F(Tmax_C):
    "Function converts temp in c to f"
    Tmax_F = (Tmax_C * (9/5)) + 32
    
    return Tmax_F

In [ ]:
def steadman_hi(Tmax_F, RH):
    "Simple heat index calculation"
    
    HI_steadman = 0.5 * (Tmax_F + 61.0 + ((Tmax_F-68.0)*1.2) + (RH*0.094))
    
    HI_steadman = (HI_steadman + Tmax_F) / 2
    
    return HI_steadman

In [ ]:
def rothfusz_hi(HI_steadman, Tmax_F, RH):
    
    "Heat Index applied to Steadman's heat index >80F"
    
    if (HI_steadman > 80):
    
        HI_rothfusz = -42.379 + 2.04901523*Tmax_F + 10.14333127*RH - .22475541*Tmax_F*RH - .00683783*Tmax_F*Tmax_F - .05481717*RH*RH + .00122874*Tmax_F*Tmax_F*RH + .00085282*Tmax_F*RH*RH - .00000199*Tmax_F*Tmax_F*RH*RH
    
    if (RH < 13) & (Tmax_F > 80) & (Tmax_F < 112):
        adjustment = ((13-RH)/4)*math.sqrt((17-abs(Tmax_F-95))/17)
        HI_rothfusz = HI_rothfusz - adjustment 
    
    "If the RH is greater than 85% and the temperature is between 80 and 87 degrees F" 
    if (RH > 85) & (Tmax_F > 80) & (Tmax_F < 87):
        adjustment = ((RH-85)/10) * ((87-Tmax_F)/5)
        HI_rothfusz = HI_rothfusz + adjustment 
    
    return HI_rothfusz

In [ ]:
def make_hi(Tmax_F, RH):
    "Calculates the heat index for a CHIRTS Tmax value"
    Tmax_F = C_to_F(Tmax_C)
#     print('F is ', Tmax_F)
#     print('H is ', RH)
    HI_steadman = steadman_hi(Tmax_F, RH)
#     print('HI_steadman is ', HI_steadman)
    HI_rothfusz = rothfusz_hi(HI_steadman, Tmax_F, RH)
#     print('HI_rothfusz is ', HI_rothfusz)
    
    return HI_rothfusz

In [ ]:
DIR_RH = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-RH/'
DIR_RAW = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-TEMP/'

In [ ]:
FN_RH = 'GHS-Tmax-RH_1983.csv'
FN_RAW = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
RH = pd.read_csv(DIR_RH+FN_RH)
RAW = pd.read_csv(DIR_RAW+FN_RAW)

In [ ]:
RH = csv_to_xr(DIR_RH+FN_RH, time_dim = 'date', space_dim = 'ID_HDC_G0')

In [ ]:
df_temp_id = RAW['ID_HDC_G0'] # get IDs
df_temp = RAW.iloc[:,3:] # get only temp columns
df_temp.index = df_temp_id # set index values
df_temp_drop = df_temp.dropna() # Drop cities w/ no temp record 
print(len(df_temp_drop))

In [ ]:
df_temp_drop.head()

In [ ]:
df_RH_id = RH['ID_HDC_G0'] # get IDs
df_RH = RH.iloc[:,3:] # get only temp columns
df_RH.index = df_RH_id # set index values
df_RH_drop = df_RH.dropna() # Drop cities w/ no temp record 
print(len(df_RH_drop))

In [ ]:
# try np arrays
# temp_arr = df_temp_drop.to_numpy()
# rh_arr = df_RH_drop.to_numpy()

# Make them into a data array
temp_xr_da = xr.DataArray(df_temp_drop, coords=[df_temp_drop.index, df_temp_drop.columns], 
                            dims=['ID_HDC_G0', 'date'])
RH_xr_da = xr.DataArray(df_RH_drop, coords=[df_RH_drop.index, df_RH_drop.columns], 
                            dims=['ID_HDC_G0', 'date'])

In [ ]:
out = hi(temp_xr_da, RH_xr_da, 'C') # STILL THROWS ERROR FIX IT 

In [ ]:
out

# Leap Year Load Issue